In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

from subprocess import check_output
print(check_output(["ls", "../input/digit-recognizer/"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

In [ ]:
train=pd.read_csv("../input/digit-recognizer/train.csv")
test=pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
train=np.asarray(train)
test=np.asarray(test)

In [ ]:
train_dup=train

In [ ]:
def bitmap_conversion(data,threshold):
    for i in range(len(data)):
            for j in range(1,785):
                if(data[i][j]>=threshold):
                    data[i][j]=1
                else:
                    data[i][j]=0
    return data

In [ ]:
def bitmap_test_conversion(data,threshold):
    for i in range(len(data)):
            for j in range(len(data[0])):
                if(data[i][j]>=threshold):
                    data[i][j]=1
                else:
                    data[i][j]=0
    return data

In [ ]:
def bitmap_threshold(data,threshold,iterations):
    temp1=[]
    temp2=[]
    avg=threshold
    while(iterations>0):
        for i in range(len(data)):
            for j in range(len(data[0])):
                if(data[i][j]>=avg):
                    temp1.append(data[i][j])
                else:
                    temp2.append(data[i][j])
        avg1=average(temp1)
        avg2=average(temp2)
        avg=(avg1+avg2)/2
        iterations-=1
    return avg

In [ ]:
def average(temp_list):
    return round(np.sum(temp_list)/len(temp_list))

In [ ]:
threshold=bitmap_threshold(train,128,5)
X=bitmap_conversion(train,threshold)


In [ ]:
threshold

In [ ]:
X_test=bitmap_test_conversion(test,threshold)

In [ ]:
def vec_to_matrix(data,dset="train"):
    temp=np.zeros((28,28))
    count=0
    if(dset=="train"):
        k=1
        for i in range(1,785,28):
            for j in range(0,28):
                temp[count][j]=data[k]
                k+=1
            count+=1
    elif(dset=="test"):
        k=1
        for i in range(0,784,28):
            for j in range(0,28):
                temp[count][j]=data[k]
                k+=1
            count+=1
    return temp

In [ ]:
def add(temp,r,c):
    res=0
    for i in range(r,r+7):
        for j in range(c,c+7):
            res+=temp[i][j]
    return res

In [ ]:
def zoning(data,dset="train"):
    features=np.zeros((train.shape[0],16))
    if(dset=="train"):
        for i in range(len(data)):
            temp=vec_to_matrix(data[i],dset)
            r=0
            count=0
            for j in range(0,4):
                c=0
                for k in range(0,4):
                    features[i][count]=add(temp,r,c)
                    count+=1
                    c+=7
                r+=7
            features[i]/=13
    elif(dset=="test"):
        for i in range(len(data)):
            print(data[i].shape)
            temp=vec_to_matrix(data[i],dset)
            r=0
            count=0
            for j in range(0,4):
                c=0
                for k in range(0,4):
                    features[i][count]=add(temp,r,c)
                    count+=1
                    c+=7
                r+=7
            features[i]/=13
    return features

In [ ]:
def gety_labels(Y):
    temp=np.zeros((Y.shape[0],10))
    for i in range(Y.shape[0]):
        k=Y[i][0]
        temp[i][k]=1
    return temp

In [ ]:
X_train=zoning(X,"train")
Y_train=gety_labels(train)


In [ ]:
X_test=zoning(X_test,"test")

In [ ]:
def activation(z,act="sigmoid"):
    if(act=="sigmoid"):
        return (1/(1+np.exp(-z)))
    elif(act=="relu"):
        return np.maximum((0.01*z),z)
    elif(act=="tanh"):
        return np.tanh(z)

In [ ]:
def layer_sizes(X,Y):
    n_x=X.shape[1]
    n_h=10
    n_y=Y.shape[1]
    return (n_x,n_h,n_y)

In [ ]:
def initialize_parameters(n_x,n_h,n_y):
    W1=np.random.randn(n_h,n_x)*0.01
    b1=np.zeros((n_h,1))
    W2=np.random.randn(n_y,n_h)*0.01
    b2=np.zeros((n_y,1))
    
    parameters={"W1":W1,
               "b1":b1,
               "W2":W2,
               "b2":b2}
    return parameters

In [ ]:
def forward_prop(X,parameters):
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    Z1 = np.dot(W1,X.T)+b1
    A1 = activation(Z1,"relu")
    Z2 = np.dot(W2,A1)+b2
    A2 = activation(Z2,"sigmoid")
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [ ]:
def accuracy(A2,Y_train):
    y_hat=A2.T
    res=0
    for i in range(len(Y_train)):
        maxm=max_index(y_hat[i])
        if(Y_train[i][maxm]==1):
            res+=1
    print(res/420)

In [ ]:
def max_index(arr):
    maxm=arr[0]
    for i in range(len(arr)):
        if(maxm<=arr[i]):
            maxm=arr[i]
            index=i
    return index

In [ ]:
def compute_cost(A2,Y,parameters):
    m = Y.shape[0] 
    logprobs = np.sum(np.sum(np.multiply(np.log(A2),Y.T)+np.multiply(np.log(1-A2),(1-Y).T)))
    logprobs = np.squeeze(logprobs) 
    cost = -(logprobs/m)
    return cost

In [ ]:
def g1prime(A,Z):
    return (np.divide(A,Z))

In [ ]:
def back_prop(parameters,cache,Y,X):
    
    m=Y.shape[0]
    W1=parameters["W1"]
    W2=parameters["W2"]
    A1=cache["A1"]
    A2=cache["A2"]
    Z1=cache["Z1"]
    
    dZ2=A2-(Y.T)
    dW2=(1/m)*np.dot(dZ2,A1.T)
    db2=(1/m)*np.sum(dZ2,axis=1,keepdims=True)
    dZ1=np.multiply(np.dot(W2.T,dZ2),g1prime(A1,Z1))
    dW1=(1/m)*np.dot(dZ1,X)
    db1=(1/m)*np.sum(dZ1,axis=1,keepdims=True)
    
    grads={"dW1":dW1,
          "dW2":dW2,
          "db1":db1,
          "db2":db2}
    return grads

In [ ]:
def update_parameters(parameters,grads,alpha=1.2):
    W1=parameters["W1"]
    W2=parameters["W2"]
    b1=parameters["b1"]
    b2=parameters["b2"]
    
    dW1=grads["dW1"]
    dW2=grads["dW2"]
    db1=grads["db1"]
    db2=grads["db2"]
    
    W1=W1-(alpha*dW1)
    W2=W2-(alpha*dW2)
    b1=b1-(alpha*db1)
    b2=b2-(alpha*db2)
    
    parameters={"W1":W1,
               "b1":b1,
               "W2":W2,
               "b2":b2}
    
    return parameters

In [ ]:
def nn(X_train,Y_train,n_h,iterations=1000):
    n_x,n_h,n_y=layer_sizes(X_train,Y_train)
    parameters=initialize_parameters(n_x,n_h,n_y)
    costs=[]
    k=0
    for i in range(iterations):
        A2,cache=forward_prop(X_train,parameters)
        cost=compute_cost(A2,Y_train,parameters)
        grads=back_prop(parameters,cache,Y_train,X_train)
        parameters=update_parameters(parameters,grads,4)
        if(i%1000==0):
            print(cost)
            costs.append(cost)
            k+=1
    return parameters,costs

In [ ]:
parameters,costs=nn(X_train,Y_train,10,10000)

In [ ]:
costs[5]

In [ ]:
import matplotlib.pyplot as plt
x=[1,2,3,4,5,6,7,8,9,10]
plt.plot(x, costs)
plt.show()

In [ ]:
A2,cache=forward_prop(X_train,parameters)

In [ ]:
accuracy(A2,Y_train)

In [ ]:
A2_test,cache_test=forward_prop(X_test,parameters)